**SVM Example**
<img src="svm_ex.png" alt="Drawing" style="width: 400px;"/>


**A hyperplane in space is defined as**
<font size="3">
$$\mathbf{w}^T\mathbf{x} + b = 0$$ 
</font>
**SVM Margin**
<font size="3">
$$margin = \underset{x}{min}\; \frac{y_n(\mathbf{w}^Tx_n+b)}{{\left \| \mathbf{w} \right \|}_2}$$

_SVM optimization is the problem of finding $\mathbf{w}$ and $b$ such that $margin$ is maximized._
\begin{equation}
\tag{1}
(\mathbf{w},b) = arg\;\underset{\mathbf{w},b}{max}\left \{ \underset{n}{min}\;\frac{y_n(\mathbf{w}^Tx_n+b)}{{\left \| \mathbf{w} \right \|}_2} \right \}= arg\;\underset{\mathbf{w},b}{max}\left \{\frac{1}{{{\left \| \mathbf{w} \right \|}_2}} \underset{n}{min}\;y_n(\mathbf{w}^Tx_n+b) \right \}
\end{equation}

Note that: $\forall n, \; y_n(\mathbf{w}^Tx_n + b) \geq 1$
</font>

<font size="3">
Eq. (1) can be turned to a constrained optimization problem as follows:
\begin{equation}
\tag{2}
(\mathbf{w},b) = arg\;\underset{\mathbf{w},b}{min} \frac{1}{2} {\left \| \mathbf{w} \right \|}_2^2
\end{equation}
subject to
$$1-y_n(\mathbf{w}^Tx_n+b) \leq 0,\; \forall n=1,2,...,N$$
</font>

### Lagrangian of SVM

<font size="3">
Lagrangian of (2) is:
\begin{equation}
\tag{3}
\mathbf{L}(\mathbf{w},b,\lambda) = \frac{1}{2} {\left \| \mathbf{w} \right \|}_2^2 \;+\; \sum_{n=1}^{N}\lambda_n(1-y_n(\mathbf{w}^Tx_n+b))
\end{equation}
with
$\lambda=[\lambda_1,\lambda_2,...,\lambda_N]^T$ and $\lambda_n \geq 0,\;\forall n=1,2,...,N$
</font>

### SVM Dual Lagrangian function

<font size="3">
\begin{equation}
\tag{4}
g(\lambda) = \underset{\mathbf{w},b}{min}\;\mathbf{L}(\mathbf{w},b,\lambda)
\end{equation}
with $$\lambda\succeq0$$

It's equivalent:
\begin{equation}
\nabla_{w,b,\lambda} \mathbf{L}(\mathbf{w},b,\lambda) = 0
\end{equation}

\begin{cases}
\frac{\partial\mathbf{L}(\mathbf{w}, b, \lambda)}{\partial_{w}} = \mathbf{w} - \sum_{n=1}^{N}\lambda_n\\
\frac{\partial\mathbf{L}(\mathbf{w}, b, \lambda)}{\partial_{b}}
\end{cases}

By solving (4), we get
\begin{equation}
\tag{5}
g(\lambda)=\sum_{n=1}^{N}\lambda_n - \frac{1}{2}\sum_{n=1}^{N}\sum_{m=1}^{N}\lambda_n\lambda_my_ny_m\mathbf{x}_n^T\mathbf{x}_m
\end{equation}
</font>

**Matrix Representation**
<font size="3">
Set $\mathbf{V}=[y_1x_1,y_2x_2,...,y_Nx_N]$
and vector $\mathbf{1}=[1,1,...,1]^T$, $g(\lambda)$ in (5) can then be represented as:
\begin{equation}
\tag{6}
g(\lambda)=-\frac{1}{2}\lambda^T\mathbf{V}^T\mathbf{V}\lambda + \mathbf{1}^T\lambda
\end{equation}
</font>

<font size="3">
Let $\mathbf{K}=\mathbf{V}^T\mathbf{V}$, we can prove that $\mathbf{K} \succeq 0$

(6) can be then represented as:
\begin{equation}
\tag{7}
g(\lambda) = -\frac{1}{2}\lambda^T\mathbf{K}\lambda + \mathbf{1}^T\lambda
\end{equation}

$g(\lambda)$ is a concave function
</font>

### SVM Dual Lagrangian problem

<font size="3">
We want to solve the dual problem of (7), which is:
\begin{equation}
\tag{8}
\lambda=arg\; \underset{\lambda}{max}\;g(\lambda)
\end{equation}

subject to: $$\lambda \succeq 0$$
$$\sum_{n=1}^{N}\lambda_ny_n=0$$
</font>

<font size="3">
We can prove that
\begin{equation}
\tag{9}
b = \frac{1}{N_{S}}\sum_{n\in S}(y_n-\mathbf{w}^Tx_n) = \frac{1}{N_{S}}\sum_{n\in S} \left ( y_n - \sum_{m \in S} \lambda_my_mx_m^Tx_n \right )
\end{equation}
    
</font>

<font size="3">
\begin{equation}
\tag{10}
\mathbf{w} = \sum_{m \in S} \lambda_my_mx_m
\end{equation}
</font>

### Self-programming


In [1]:
from __future__ import print_function
import numpy as np 
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
np.random.seed(22)

#Generate simulation data
means = [[2, 2], [4, 2]]
cov = [[.3, .2], [.2, .3]]
N = 10
X0 = np.random.multivariate_normal(means[0], cov, N) # class 1
X1 = np.random.multivariate_normal(means[1], cov, N) # class -1 
X = np.concatenate((X0.T, X1.T), axis = 1) # all data 
y = np.concatenate((np.ones((1, N)), -1*np.ones((1, N))), axis = 1) # labels 

##### Solving equation (8) by using CVXOPT

In [8]:
from cvxopt import matrix, solvers
# build K
V = np.concatenate((X0.T, -X1.T), axis = 1)

K = matrix(V.T.dot(V)) # see definition of V, K near eq (6)
p = matrix(-np.ones((2*N, 1))) # all-one vector 
# build A, b, G, h 

G = matrix(-np.eye(2*N)) # for all lambda_n >= 0
h = matrix(np.zeros((2*N, 1)))
A = matrix(y) # the equality constrain is actually y^T lambda = 0
print(A)
b = matrix(np.zeros((1, 1))) 
print(b)
solvers.options['show_progress'] = False
sol = solvers.qp(K, p, G, h, A, b)

l = np.array(sol['x'])
print('lambda = ')
print(l.T)

[ 1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00 ... ]

[ 0.00e+00]

lambda = 
[[8.54018321e-01 2.89132533e-10 1.37095535e+00 6.36030818e-10
  4.04317408e-10 8.82390106e-10 6.35001881e-10 5.49567576e-10
  8.33359230e-10 1.20982928e-10 6.86678649e-10 1.25039745e-10
  2.22497367e+00 4.05417905e-09 1.26763684e-10 1.99008949e-10
  2.13742578e-10 1.51537487e-10 3.75329509e-10 3.56161975e-10]]


In [15]:
epsilon = 1e-6 # just a small number, greater than 1e-9
print(np.where(l > epsilon))
S = np.where(l > epsilon)[0]
print(S)

VS = V[:, S]
print(V)
print(VS)
XS = X[:, S]
yS = y[:, S]
lS = l[S]
# calculate w and b
w = VS.dot(lS)
b = np.mean(yS.T - w.T.dot(XS))

print('w = ', w.T)
print('b = ', b)

(array([ 0,  2, 12]), array([0, 0, 0]))
[ 0  2 12]
[[ 2.37319011  1.51261889  2.4696794   1.78736889  1.81231157  2.03717355
   1.53790057  2.29312867  1.38805594  1.57279694 -3.42746579 -4.24760864
  -3.33595491 -3.69420104 -4.53897645 -3.3071994  -4.13924705 -4.47383468
  -4.00512009 -4.28205624]
 [ 1.71875981  1.40558943  2.02144973  1.29380961  1.56119497  1.93397133
   1.87434722  2.76537389  1.86419379  0.90707347 -0.71254431 -2.39846497
  -1.61731637 -1.94273986 -2.54957308 -0.19362396 -2.09561534 -2.41269466
  -1.89290099 -1.79675607]]
[[ 2.37319011  2.4696794  -3.33595491]
 [ 1.71875981  2.02144973 -1.61731637]]
w =  [[-2.00984381  0.64068336]]
b =  4.668560633868063


### Using scikit-learn

In [34]:
from sklearn.svm import SVC

y1 = y.reshape((2*N,))
X1 = X.T # each sample is one row
clf = SVC(kernel = 'linear', C = 1e5) # just a big number 

clf.fit(X1, y1) 

w = clf.coef_
b = clf.intercept_
print('w = ', w)
print('b = ', b)

w =  [[-2.00971102  0.64194082]]
b =  [4.66595309]
